In [1]:
import numpy as np
import pandas as pd
from dgutils.pandas import add_column, add_columns

/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import pandas as pd
sns.set(color_codes=True)
import cufflinks as cf
cf.go_offline()
cf.set_config_file(theme='ggplot')
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.figure_factory as ff
init_notebook_mode(connected=True)

/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/scipy/special/__init__.py:641: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  from ._ufuncs import *
/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  from ._solve_toeplitz import levinson
/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  from ._decomp_update import *
/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:6: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 21

In [3]:
from utils import arr2db, EvalMetric, forna_url

Using TensorFlow backend.
/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/tensorflow/python/framework/tensor_util.py:32: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/scipy/ndimage/measurements.py:36: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [4]:
# df = pd.read_pickle('result/test_7.pkl')
# df = pd.read_pickle('result/prediction.rfam.2019_09_07_1.sample20.pkl')
# df = pd.read_pickle('result/prediction.rfam.2019_09_08_1.sample20.pkl')

# df = pd.read_pickle('result/prediction.s_test.2019_09_08_1.sample20.pkl')

# df = pd.read_pickle('result/prediction.rfam.2019_09_11_1.sample20.pkl')

df = pd.read_pickle('result/prediction.rfam.2019_09_12_1.sample20.pkl')

In [5]:
def pick_metric(sensitivity, ppv, logps):
    idx = np.argmax(logps)
    return sensitivity[idx], ppv[idx]

In [6]:
def pick_metric_2(sensitivity, ppv, pred_idx):
    # pick my max number of base pairs -> crude approximation of energy?
    # this doesn't work well
    tmp = [len(x[0]) for x in pred_idx]
    idx = np.argmax(tmp)
    return sensitivity[idx], ppv[idx]

In [7]:
def pick_metric_3(sensitivity, ppv, fe):
    idx = np.argmin(fe)
    return sensitivity[idx], ppv[idx]

In [8]:
df = add_columns(df, ['sensitivity_max', 'sensitivity_median'], 
                     ['sensitivity'], lambda x: (np.nanmax(x), np.nanmedian(x)))
df = add_columns(df, ['ppv_max', 'ppv_median'],
                    ['ppv'], lambda x: (np.nanmax(x), np.nanmedian(x)))
# logp
df = add_columns(df, ['sensitivity_mp', 'ppv_mp'], 
                       ['sensitivity', 'ppv', 'logp'], pick_metric)
# max base pairs
df = add_columns(df, ['sensitivity_mb', 'ppv_mb'], 
                       ['sensitivity', 'ppv', 'pred_idx'], pick_metric_2)

# predicted fe
df = add_columns(df, ['sensitivity_mfe', 'ppv_mfe'], 
                       ['sensitivity', 'ppv', 'fe'], pick_metric_3)

In [9]:
df.describe()

,len,ppv_max,ppv_mb,ppv_median,ppv_mfe,ppv_mp,sensitivity_max,sensitivity_mb,sensitivity_median,sensitivity_mfe,sensitivity_mp
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,136.298013,0.687173,0.534819,0.562719,0.538270,0.593518,0.720704,0.643149,0.606524,0.647485,0.627575
std,102.036451,0.266043,0.290747,0.293887,0.291962,0.300105,0.271661,0.313872,0.312826,0.316150,0.314152
min,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,67.000000,0.500000,0.336966,0.339080,0.336966,0.375672,0.541958,0.445489,0.344582,0.445489,0.429167
50%,104.000000,0.727273,0.534884,0.588235,0.539216,0.611111,0.781250,0.687500,0.666667,0.687500,0.636364
75%,158.500000,0.916667,0.760889,0.775972,0.774194,0.855655,1.000000,0.944947,0.868116,0.955005,0.936105
max,568.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
def make_arr(l, one_idx):
    x = np.zeros((l, l))
    x[one_idx] = 1
    return x

In [11]:
idx = np.random.randint(0, len(df))
# idx = 185  # debug shortest seq in s_test
row = df.iloc[idx]

seq = row['seq']
target = make_arr(len(seq), row['one_idx'])
pred_idxes = row['pred_idx']
logp = row['logp']
pfe = row['fe']

print len(seq), seq

55 UCAUUUGUAAGUCGCUUUGGAUAAAAGCGUCUGCUAAAUGACUAAAUGUAAAUGU


In [12]:
row

f_measure             [0.8000000000000002, 0.8000000000000002, 0.800...
fe                    [-0.27410212, -0.27410212, -0.27410212, -0.274...
len                                                                  55
logp                  [-0.0028859312630800105, -0.002886050461484046...
one_idx               ([0, 1, 2, 3, 4, 5, 6, 7, 8, 12, 13, 14, 15, 1...
ppv                   [0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, ...
ppv_max                                                             0.8
ppv_mb                                                              0.8
ppv_median                                                          0.8
ppv_mfe                                                             0.8
ppv_mp                                                              0.8
pred_idx              [([0, 1, 2, 3, 4, 5, 6, 9, 10, 12, 13, 14, 15,...
sensitivity           [0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, ...
sensitivity_max                                                 

In [13]:
axis_label = ["{}{}".format(i+1, x) for i, x in enumerate(seq)]
# axis_label = ["{}{}".format(i, x) for i, x in enumerate(seq)]

fig = go.Figure(data=[go.Heatmap(
                   z=target,
                   x=axis_label,
                   y=axis_label,
    reversescale=True,
    colorscale='Greys')])

# layout = dict(
#     title='target',
#     xaxis=dict(title='seq'),
#     yaxis=dict(title='seq', autorange='reversed'),
#      height=800,
#     width=800,
#     )
layout = dict(
    title='target',
#     xaxis=dict(title='seq'),
    yaxis=dict(autorange='reversed'),
     height=800,
    width=800,
    )
fig['layout'].update(layout)
iplot(fig)

In [14]:

# plt.figure(figsize=(12, 12))
# tb = plt.table(cellText=target.astype(int), loc=(0,0), cellLoc='center')

# tc = tb.properties()['child_artists']
# for cell in tc: 
#     cell.set_height(1./len(seq))
#     cell.set_width(1./len(seq))

# ax = plt.gca()
# ax.set_xticks([])
# ax.set_yticks([])

In [15]:
target_db_str = arr2db(target)
print forna_url(seq, target_db_str)

http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=UCAUUUGUAAGUCGCUUUGGAUAAAAGCGUCUGCUAAAUGACUAAAUGUAAAUGU&structure=(((((((((...((((((.....))))))..))).))))))..............


In [16]:
n_sample = min(len(pred_idxes), 20)  # not plotting everything! in case there are too many


axis_label = ["{}{}".format(i, x) for i, x in enumerate(seq)]

n_rows = int(np.floor(np.sqrt(n_sample)))
n_cols = int(np.ceil(n_sample/n_rows))
fig = plotly.tools.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=range(n_sample))

# db_strs = []
df_metric = []

for idx_sample in range(n_sample):  
    _pred = make_arr(len(seq), pred_idxes[idx_sample])
    
    # dot-bracket notation
    db_str = arr2db(_pred)
#     db_strs.append(db_str)
#     print idx_sample, db_str
    print idx_sample
    print forna_url(seq, db_str)
    
    # metric
    sensitivity = EvalMetric.sensitivity(_pred, target)
    ppv = EvalMetric.ppv(_pred, target)
    f_measure = EvalMetric.f_measure(sensitivity, ppv)
    print "sensitivity: {}, ppv: {}, f measure: {}".format(sensitivity, ppv, f_measure)
    print logp[idx_sample], pfe[idx_sample]
    print ''
#     df_metric.append({
#         'idx_sample': idx_sample,
#         'seq': seq,
#         'db_str': db_str,
#         'pred': _pred,
#         'sensitivity': sensitivity,
#         'ppv': ppv,
#         'f_measure': f_measure,
#     })
    
    # set diagonal to 0.5, for visualization
    _pred[np.diag_indices_from(_pred)] = 0.5
    trace = go.Heatmap(
                   z=_pred,
                   x=axis_label,
                   y=axis_label,
    reversescale=True,
    showscale=False,
    colorscale='Greys')
    fig.append_trace(trace, idx_sample//n_cols + 1, idx_sample%n_cols + 1)
    

layout = dict(
#     title='prediction_1_step_ar',
#     title='{} {}'.format(idx_sample, db_str),
    xaxis=dict(showticklabels=False, showgrid=False),
    yaxis=dict(autorange='reversed', showticklabels=False, showgrid=False),
     height=1200,
    width=1000,
    )
layout['annotations'] = []
for idx_sample in range(1, n_sample):
    layout["xaxis{}".format(idx_sample+1)]=dict(showticklabels=False, showgrid=False)
    layout["yaxis{}".format(idx_sample+1)]=dict(autorange='reversed', showticklabels=False, showgrid=False)
#     layout['annotations'].append({"text": db_strs[idx_sample]})
fig['layout'].update(layout)
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]    [ (1,3) x3,y3 ]    [ (1,4) x4,y4 ]    [ (1,5) x5,y5 ]  
[ (2,1) x6,y6 ]    [ (2,2) x7,y7 ]    [ (2,3) x8,y8 ]    [ (2,4) x9,y9 ]    [ (2,5) x10,y10 ]
[ (3,1) x11,y11 ]  [ (3,2) x12,y12 ]  [ (3,3) x13,y13 ]  [ (3,4) x14,y14 ]  [ (3,5) x15,y15 ]
[ (4,1) x16,y16 ]  [ (4,2) x17,y17 ]  [ (4,3) x18,y18 ]  [ (4,4) x19,y19 ]  [ (4,5) x20,y20 ]

0
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=UCAUUUGUAAGUCGCUUUGGAUAAAAGCGUCUGCUAAAUGACUAAAUGUAAAUGU&structure=(((((((..((.((((((.....)))))).))..)))))))..............
sensitivity: 0.8, ppv: 0.8, f measure: 0.8
-0.0028859312630800105 -0.27410212

1
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=UCAUUUGUAAGUCGCUUUGGAUAAAAGCGUCUGCUAAAUGACUAAAUGUAAAUGU&structure=(((((((..((.((((((.....)))))).))..)))))))..............
sensitivity: 0.8, ppv: 0.8, f measure: 0.8
-0.0028860504614840465 -0.27410212

2
http://nibiru.tbi.univie.ac.at/forna/fo

/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/plotly/basedatatypes.py:1222: DeprecationWarning:

The append_trace method is deprecated and will be removed in a future version.
Please use the add_trace method with the row and col parameters.


